In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import style
style.use("fivethirtyeight")
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler

In [2]:
pima=pd.read_csv("diabetes.csv")
pima.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
scalar=StandardScaler()
x=np.array(pima.drop("Outcome",axis=1))
#x=scalar.fit_transform(x)
y=np.array(pima["Outcome"])

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=36)

In [6]:
x_train=x_train.reshape(x_train.shape[1],x_train.shape[0])
x_test=x_test.reshape(x_test.shape[1],x_test.shape[0])
y_train=y_train.reshape(1,y_train.shape[0])
y_test=y_test.reshape(1,y_test.shape[0])

In [7]:
class SANN:
    
    def __init__(self,shapes,layers,neurons,activation_function):
        self.shapes=shapes
        self.layers=layers
        self.neurons=neurons
        self.activation_function=activation_function
        
    def layersizes(self):
        
        self.sizes=[]
        self.sizes.append(self.shapes[0])
        for i in self.neurons:
            self.sizes.append(i)
        self.sizes.append(self.shapes[1])
        
        return self.sizes
    
    def initializing_parameters(self):
        
        layer_dims=self.layersizes()
        self.parameters={}
        l=len(layer_dims)-1
        
        for j in range(1,l):
            self.parameters["W"+str(j)]=np.random.randn(layer_dims[j],layer_dims[j-1])*0.01
            self.parameters["b"+str(j)]=np.zeros((layer_dims[j],1))
        
        return self.parameters
    
    def forward_propagation(self,X,parameters):
        
        cache={"A0":X}
        
        
        for k in range(1,self.layers+1):
            
            cache["Z"+str(k)]=np.dot(parameters["W"+str(k)],cache["A"+str(k-1)])+parameters["b"+str(k)]
            
            if self.activation_function[k-1]=="tanh":
                cache["A"+str(k)]=np.tanh(cache["Z"+str(k)])
                
            elif self.activation_function[k-1]=="sigmoid":
                cache["A"+str(k)]=1/(1+np.exp(-cache["Z"+str(k)]))
                
            elif self.activation_function[k-1]=="relu":
                cache["A"+str(k)]=np.maximum(0,cache["Z"+str(k)])
                
        return cache["A"+str(self.layers)],cache
            
            
    def compute_cost(self,Af,Y):
        
        m=Y.shape[1]

        cost=-np.sum((np.dot(np.log(Af),Y.T)+np.dot(np.log(1-Af),(1-Y).T)))/m

        cost=float(np.squeeze(cost))

        return cost
    
    def backward_propagation(self,X,Y,cache,parameters):
        
        m=X.shape[1]
        
        active=self.activation_function[:-1]
        derivatives={}
        grads={}
        
        derivatives["dZ"+str(self.layers)]=np.subtract(cache["A"+str(self.layers)],Y)
        grads["dW"+str(self.layers)]=np.dot(derivatives["dZ"+str(self.layers)],cache["A"+str(self.layers-1)].T)
        grads["db"+str(self.layers)]=(np.sum(derivatives["dZ"+str(self.layers)],axis=1,keepdims=True))/m
        
        for b in range(self.layers-1,0,-1):
            if active[-1]=="tanh":
                derivatives["dZ"+str(b)]=np.dot(parameters["W"+str(b+1)].T,derivatives["dZ"+str(b+1)])*(1-np.power(cache["A"+str(b)],2))
                grads["dW"+str(b)]=np.dot(derivatives["dZ"+str(b)],cache["A"+str(b-1)].T)/m
                active.pop()
            elif active[-1]=="sigmoid":
                dr=1/(1+np.exp(-cache["A"+str(b)]))
                derivatives["dZ"+str(b)]=np.dot(parameters["W"+str(b+1)].T,derivatives["dZ"+str(b+1)])*(dr*(1-dr))
                grads["dW"+str(b)]=np.dot(derivatives["dZ"+str(b)],cache["A"+str(b-1)].T)/m
                active.pop()
            elif active[-1]=="relu":
                if (cache["A"+str(b)]<0).any():
                    t=0
                else:
                    t=1
                derivatives["dZ"+str(b)]=np.dot(parameters["W"+str(b+1)].T,derivatives["dZ"+str(b+1)])*t
                grads["dW"+str(b)]=np.dot(derivatives["dZ"+str(b)],cache["A"+str(b-1)].T)/m
                active.pop()
                
            grads["db"+str(b)]=(np.sum(derivatives["dZ"+str(b)],axis=1,keepdims=True))/m
        
        return grads
    
    
    def update_parameters(self,parameters,grads,lr):
        
        new_parameters={}
        
        for i in range(1,self.layers+1):
            
            new_parameters["W"+str(i)]=parameters["W"+str(i)]-lr*grads["dW"+str(i)]
            new_parameters["b"+str(i)]=parameters["b"+str(i)]-lr*grads["db"+str(i)]
            
        return new_parameters
     
    def fit(self,training_set,validation_set,epochs,learning_rate,verbose=False):
        
        self.X=training_set[0]
        self.Y=training_set[1]
        self.cost_error=[]
        self.accuracy_score=[]
        self.weights=self.initializing_parameters()
        
        for j in range(epochs):
            
            Afinal,cache=self.forward_propagation(self.X,self.weights)
            
            cost=self.compute_cost(Afinal,self.Y)
            
            grads=self.backward_propagation(self.X,self.Y,cache,self.weights)
            
            self.weights=self.update_parameters(self.weights,grads,learning_rate)
            
            s=1000
            if (j%s==0):
                y_pred=self.predictions(self.X)
                acc=accuracy_score(self.Y[0],y_pred)*100
                self.cost_error.append(cost)
                self.accuracy_score.append(acc)
                print(f"cost at {j} iteration is ====> {cost} and accuracy score is {acc}")
    
        return self.cost_error,self.accuracy_score
    
    def predictions(self,X):
            Afinal,cache=self.forward_propagation(X,self.weights)
            predictions = []
            for i in range(len(Afinal[0])):
                if (Afinal[0][i]>0.5).any():
                    predictions.append(1)
                else:
                    predictions.append(0)
            predictions=np.array(predictions)
            return predictions
                    

In [8]:
shapes=[x_train.shape[0],y_train.shape[0]]
sample=SANN(shapes,5,[128,64,32,16,1],["tanh","tanh","tanh","tanh","sigmoid"])
loss,accuracy=sample.fit(training_set=[x_train,y_train],
             validation_set=[x_test,y_test],
             epochs=20000,learning_rate=0.005)

cost at 0 iteration is ====> 0.6931491791617985 and accuracy score is 64.40972222222221
cost at 1000 iteration is ====> 0.6509979220630847 and accuracy score is 64.40972222222221
cost at 2000 iteration is ====> 0.6509667638775476 and accuracy score is 64.40972222222221
cost at 3000 iteration is ====> 0.6509182348144056 and accuracy score is 64.40972222222221
cost at 4000 iteration is ====> 0.6508251957948235 and accuracy score is 64.40972222222221
cost at 5000 iteration is ====> 0.6506058587911134 and accuracy score is 64.40972222222221
cost at 6000 iteration is ====> 0.6498942774772958 and accuracy score is 64.40972222222221
cost at 7000 iteration is ====> 0.6449215067500172 and accuracy score is 64.40972222222221
cost at 8000 iteration is ====> 0.5710040477215946 and accuracy score is 72.04861111111111
cost at 9000 iteration is ====> 0.4888972683640264 and accuracy score is 77.60416666666666
cost at 10000 iteration is ====> 0.42555995496659055 and accuracy score is 80.90277777777779


In [9]:
plt.ion()
%matplotlib qt
for i in range(1,len(loss)):
    x=range(0,i)
    y=loss[0:i]
    plt.plot(x,y)
    plt.ylabel("cost function error")
    plt.xlabel("cost function for every 1000 iterations")
    plt.title(str(i))
    plt.draw()
    plt.pause(0.1)
plt.show(block=True)

In [10]:
plt.ion()
%matplotlib qt
for i in range(1,len(accuracy)):
    x=range(0,i)
    y=accuracy[0:i]
    plt.plot(x,y)
    plt.ylabel("training accuracy")
    plt.xlabel("training accuracy 1000 iterations")
    plt.title(str(i))
    plt.draw()
    plt.pause(0.1)
plt.show(block=True)

In [11]:
y_pred=sample.predictions(x_test)
print(confusion_matrix(y_test[0],y_pred))
print(classification_report(y_test[0],y_pred))
print(accuracy_score(y_test[0],y_pred))

[[80 49]
 [39 24]]
              precision    recall  f1-score   support

           0       0.67      0.62      0.65       129
           1       0.33      0.38      0.35        63

    accuracy                           0.54       192
   macro avg       0.50      0.50      0.50       192
weighted avg       0.56      0.54      0.55       192

0.5416666666666666
